In [4]:
!curl https://topcs.blob.core.windows.net/public/FlightData.csv -o flightdata.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 12 1552k   12  191k    0     0   113k      0  0:00:13  0:00:01  0:00:12  113k
100 1552k  100 1552k    0     0   634k      0  0:00:02  0:00:02 --:--:--  634k


Esse comando baixa um arquivo CSV do Armazenamento de Blobs do Azure e salva-o usando o nome flightdata.csv.

In [5]:
import pandas as pd

df = pd.read_csv('flightdata.csv')
df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,...,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,Unnamed: 25
0,2016,1,1,1,5,DL,N836DN,1399,10397,ATL,...,2143,2102.0,-41.0,0.0,0.0,0.0,338.0,295.0,2182.0,NaN
1,2016,1,1,1,5,DL,N964DN,1476,11433,DTW,...,1435,1439.0,4.0,0.0,0.0,0.0,110.0,115.0,528.0,NaN
2,2016,1,1,1,5,DL,N813DN,1597,10397,ATL,...,1215,1142.0,-33.0,0.0,0.0,0.0,335.0,300.0,2182.0,NaN
3,2016,1,1,1,5,DL,N587NW,1768,14747,SEA,...,1335,1345.0,10.0,0.0,0.0,0.0,196.0,205.0,1399.0,NaN
4,2016,1,1,1,5,DL,N836DN,1823,14747,SEA,...,607,615.0,8.0,0.0,0.0,0.0,247.0,259.0,1927.0,NaN


carregar flightdata.csv, criar um DataFrame do Pandas com base nesse arquivo e exibir as primeiras cinco linhas.

O DataFrame contém informações de chegada no horário para uma grande companhia aérea dos EUA. Ele tem mais de 11.000 linhas e 26 colunas. (A saída diz "5 linhas" porque a função head do DataFrame retorna apenas as primeiras cinco linhas.) Cada linha representa um voo e contém informações como a origem, o destino, a hora de partida agendada e se o voo chegou na hora ou atrasado.

##### Para obter uma contagem de linhas e de colunas.

In [6]:
df.shape

(11231, 26)

Aqui está uma lista completa das colunas no conjunto de dados. Os horários são expressos em formato de 24 horas. Por exemplo, 1130 é igual a 11:30. e 1500 é igual a 15:00.

##### Coluna	Descrição
###### YEAR	
O ano em que o voo foi realizado
###### 
QUARTER	O trimestre em que o voo foi realizado (1-4)
###### MONTH	
O mês em que o voo foi realizado (1-12)
###### DAY_OF_MONTH	
O dia do mês em que o voo foi realizado (1-31)
###### DAY_OF_WEEK	
Dia da semana em que o voo ocorreu (1=segunda-feira, 2=terça-feira, etc.)
###### UNIQUE_CARRIER	
Código da operadora de companhia aérea (por exemplo, DL)
###### TAIL_NUM	
Número da cauda da aeronave
###### FL_NUM	
Número do voo
###### ORIGIN_AIRPORT_ID	
ID do aeroporto de origem
###### ORIGIN	
Código do aeroporto de origem (ATL, DFW, SEA, etc.)
###### DEST_AIRPORT_ID	
ID do aeroporto de destino
###### DEST	
Código do aeroporto de destino (ATL, DFW, SEA, etc.)
###### CRS_DEP_TIME	
Hora de partida agendada
###### DEP_TIME	
Hora de partida real
###### DEP_DELAY	
Número de minutos em que a partida foi atrasada
###### DEP_DEL15	0=Partida atrasada em menos de 15 minutos, 1=Partida atrasada em 15 minutos ou mais
###### CRS_ARR_TIME	
Hora de chegada agendada
###### ARR_TIME	
Hora de chegada real
###### ARR_DELAY	
Número de minutos em que a chegada do voo atrasou
###### ARR_DEL15	0=Chegou com menos de 15 minutos de atraso, 1=Chegou com 15 minutos de atraso ou mais
###### CANCELLED	0=O voo não foi cancelado, 1=O voo foi cancelado
###### DIVERTED	0=O voo não foi desviado, 1=O voo foi desviado
###### CRS_ELAPSED_TIME	
Tempo de voo agendado em minutos
###### ACTUAL_ELAPSED_TIME	
Tempo de voo real em minutos
###### DISTANCE	
Distância percorrida em milhas

O conjunto de dados inclui uma distribuição aproximadamente uniforme de datas ao longo do ano, o que é importante porque um voo saindo de Minneapolis tem menos probabilidade de atrasar devido a tempestades de inverno em julho do que em janeiro. Mas esse conjunto de dados está longe de ser "limpo" e pronto para uso. Vamos escrever um código de Pandas para limpá-lo.

Um dos aspectos mais importantes de preparação de um conjunto de dados para uso no machine learning é selecionar as colunas de "recurso" que são relevantes para o resultado que você está tentando prever, filtrando colunas que não afetam o resultado, que poderiam influenciá-lo de forma negativa ou que podem pode produzir multicolinearidade. Outra tarefa importante é eliminar valores ausentes, excluindo as linhas ou colunas que os contêm ou substituindo-os com valores significativos. Neste exercício, você elimina colunas estranhas e substitui valores ausentes nas colunas restantes.

##### Valores ausentes são uma das primeiras coisas que os cientistas de dados normalmente procuram em um conjunto de dados. Há uma maneira fácil de verificar se há valores ausentes no Pandas

In [7]:
df.isnull().values.any()

True

a saída é "True", o que indica que há pelo menos um valor ausente em algum lugar no conjunto de dados.

##### A próxima etapa é descobrir onde estão os valores ausentes

In [8]:
df.isnull().sum()

YEAR                       0
QUARTER                    0
MONTH                      0
DAY_OF_MONTH               0
DAY_OF_WEEK                0
UNIQUE_CARRIER             0
TAIL_NUM                   0
FL_NUM                     0
ORIGIN_AIRPORT_ID          0
ORIGIN                     0
DEST_AIRPORT_ID            0
DEST                       0
CRS_DEP_TIME               0
DEP_TIME                 107
DEP_DELAY                107
DEP_DEL15                107
CRS_ARR_TIME               0
ARR_TIME                 115
ARR_DELAY                188
ARR_DEL15                188
CANCELLED                  0
DIVERTED                   0
CRS_ELAPSED_TIME           0
ACTUAL_ELAPSED_TIME      188
DISTANCE                   0
Unnamed: 25            11231
dtype: int64

Para eliminar a 26ª coluna ("Sem nome: 25") que contém 11.231 valores ausentes e vizualizar o DataFrame com a coluna 26 removida:

In [9]:
df = df.drop('Unnamed: 25', axis=1)
df.isnull().sum()

YEAR                     0
QUARTER                  0
MONTH                    0
DAY_OF_MONTH             0
DAY_OF_WEEK              0
UNIQUE_CARRIER           0
TAIL_NUM                 0
FL_NUM                   0
ORIGIN_AIRPORT_ID        0
ORIGIN                   0
DEST_AIRPORT_ID          0
DEST                     0
CRS_DEP_TIME             0
DEP_TIME               107
DEP_DELAY              107
DEP_DEL15              107
CRS_ARR_TIME             0
ARR_TIME               115
ARR_DELAY              188
ARR_DEL15              188
CANCELLED                0
DIVERTED                 0
CRS_ELAPSED_TIME         0
ACTUAL_ELAPSED_TIME    188
DISTANCE                 0
dtype: int64

O DataFrame ainda contém muitos valores ausentes, mas alguns deles não são úteis porque as colunas que os contêm não são relevantes para o modelo que você está criando. O objetivo desse modelo é prever se um voo que você está pensando em reservar provavelmente chegará no horário. 

A próxima etapa filtrar o conjunto de dados para eliminar as colunas que não são relevantes para um modelo preditivo. Por exemplo, o número da cauda da aeronave provavelmente tem pouca influência na pontualidade de um voo e, no momento em que reserva uma passagem, você não tem como saber se um voo será cancelado, desviado ou atrasado. Por outro lado, a hora de partida agendada pode ter muita influência sobre a pontualidade da chegada. Por causa do sistema de hub e spoke usado pela maioria das companhias aéreas, os voos matinais tendem a chegar no horário mais frequentemente do que aqueles vespertinos ou noturnos. E em alguns dos principais aeroportos, o tráfego se acumula durante o dia, aumentando a probabilidade de que voos em horários mais próximos do fim do dia se atrasem.


##### O Pandas fornece uma maneira fácil de filtrar as colunas que você não deseja. 
A saída mostra que o DataFrame agora inclui apenas as colunas que são relevantes para o modelo e o número de valores ausentes é bastante reduzido:

In [10]:
df = df[["MONTH", "DAY_OF_MONTH", "DAY_OF_WEEK", "ORIGIN", "DEST", "CRS_DEP_TIME", "ARR_DEL15"]]
df.isnull().sum()

MONTH             0
DAY_OF_MONTH      0
DAY_OF_WEEK       0
ORIGIN            0
DEST              0
CRS_DEP_TIME      0
ARR_DEL15       188
dtype: int64

A única coluna que agora contém valores ausentes é a coluna de ARR_DEL15, que usa 0s para identificar os voos que chegaram na hora e 1s para voos que atrasaram. Use o código a seguir para mostrar as primeiras cinco linhas com valores ausentes:

In [11]:
df[df.isnull().values.any(axis=1)].head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN,DEST,CRS_DEP_TIME,ARR_DEL15
177,1,9,6,MSP,SEA,701,NaN
179,1,10,7,MSP,DTW,1348,NaN
184,1,10,7,MSP,DTW,625,NaN
210,1,10,7,DTW,MSP,1200,NaN
478,1,22,5,SEA,JFK,2305,NaN


O motivo pelo qual há valores ARR_DEL15 ausentes nessas linhas é que todos eles correspondem aos voos que foram cancelados ou desviados. Você poderia chamar dropna no DataFrame para remover essas linhas. Mas já que um voo que é cancelado ou desviado para outro aeroporto pode ser considerado "atrasado", usaremos o método fillna para substituir os valores ausentes por 1s.

###### Use o código a seguir para substituir valores ausentes na coluna ARR_DEL15 por 1s e exibir as linhas 177-184:


In [12]:
df = df.fillna({'ARR_DEL15': 1})
df.iloc[177:185]

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN,DEST,CRS_DEP_TIME,ARR_DEL15
177,1,9,6,MSP,SEA,701,1.0
178,1,9,6,DTW,JFK,1527,0.0
179,1,10,7,MSP,DTW,1348,1.0
180,1,10,7,DTW,MSP,1540,0.0
181,1,10,7,JFK,ATL,1325,0.0
182,1,10,7,JFK,ATL,610,0.0
183,1,10,7,JFK,SEA,1615,0.0
184,1,10,7,MSP,DTW,625,1.0


###### Confirme se os NaNs nas linhas 177, 179 e 184 foram substituídos por 1s, indicando que os voos chegaram atrasados

O conjunto de dados agora está "limpo", no sentido de que valores ausentes foram substituídos e a lista de colunas foi reduzida para aquelas mais relevantes para o modelo. Mas você não ainda terminou. Há mais o que fazer para preparar o conjunto de dados para uso no aprendizado de máquina.

A coluna CRS_DEP_TIME do conjunto de dados que você está usando representa horários de partida agendados. A granularidade dos números nessa coluna (que contém mais de 500 valores exclusivos) pode ter um impacto negativo na precisão em um modelo de machine learning. Isso pode ser resolvido usando uma técnica chamada compartimentalização. E se você dividir cada número nesta coluna por 100 e arredondar para baixo até o inteiro mais próximo? 1030 se tornará 10, 1925 se tornará 19 e assim por diante e você ficará com um máximo de 24 valores discretos nesta coluna. Intuitivamente, faz sentido, porque provavelmente não importa muito se um voo sai às 10:30 ou às 10:40. Mas faz muita diferença se ele sai às 10:30 ou às 17:30.

Além disso, as colunas ORIGIN e DEST do conjunto de dados contêm códigos de aeroporto que representam valores categóricos do aprendizado de máquina. Essas colunas precisam ser convertidas em colunas discretas que contém variáveis de indicador, às vezes conhecidas como variáveis "fictícias". Em outras palavras, a coluna ORIGIN, que contém cinco códigos de aeroporto, precisa ser convertida em cinco colunas, uma por aeroporto, com cada coluna contendo 0s e 1s que indicam se um voo teve origem no aeroporto que a coluna representa. A coluna DEST precisa ser tratada de maneira semelhante.

###### Neste exercício, você "compartimentalizará" os horários de partida na coluna CRS_DEP_TIME e usará o método 'Pandas get_dummies para criar colunas de indicador das colunas ORIGIN e DEST.

In [13]:
df.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN,DEST,CRS_DEP_TIME,ARR_DEL15
0,1,1,5,ATL,SEA,1905,0.0
1,1,1,5,DTW,MSP,1345,0.0
2,1,1,5,ATL,SEA,940,0.0
3,1,1,5,SEA,MSP,819,0.0
4,1,1,5,SEA,DTW,2300,0.0


Observe que a coluna CRS_DEP_TIME contém valores de 0 a 2359 representando horários em formato de 24 horas

O DataFrame com horários de partida não compartimentalizados

###### Use as instruções a seguir para compartimentalizar os horários de partida:

In [14]:
import math

for index, row in df.iterrows():
    df.loc[index, 'CRS_DEP_TIME'] = math.floor(row['CRS_DEP_TIME'] / 100)
df.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN,DEST,CRS_DEP_TIME,ARR_DEL15
0,1,1,5,ATL,SEA,19,0.0
1,1,1,5,DTW,MSP,13,0.0
2,1,1,5,ATL,SEA,9,0.0
3,1,1,5,SEA,MSP,8,0.0
4,1,1,5,SEA,DTW,23,0.0


Confirme que os números na coluna CRS_DEP_TIME agora estão no intervalo 0 a 23:

O DataFrame com horários de partida compartimentalizados

###### Agora, use as instruções a seguir para gerar colunas de indicador das colunas ORIGIN e DEST, removendo simultaneamente as colunas ORIGIN e DEST propriamente ditas:

In [15]:
df = pd.get_dummies(df, columns=['ORIGIN', 'DEST'])
df.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,CRS_DEP_TIME,ARR_DEL15,ORIGIN_ATL,ORIGIN_DTW,ORIGIN_JFK,ORIGIN_MSP,ORIGIN_SEA,DEST_ATL,DEST_DTW,DEST_JFK,DEST_MSP,DEST_SEA
0,1,1,5,19,0.0,1,0,0,0,0,0,0,0,0,1
1,1,1,5,13,0.0,0,1,0,0,0,0,0,0,1,0
2,1,1,5,9,0.0,1,0,0,0,0,0,0,0,0,1
3,1,1,5,8,0.0,0,0,0,0,1,0,0,0,1,0
4,1,1,5,23,0.0,0,0,0,0,1,0,1,0,0,0


Examine o DataFrame resultante e observe que as colunas ORIGIN e DEST foram substituídas por colunas correspondentes para os códigos de aeroporto presentes nas colunas originais. As novas colunas têm 1s e 0s que indicam se um determinado voo teve origem no ou foi destinado ao aeroporto correspondente.

# Criar modelo de machine learning

In [17]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df.drop('ARR_DEL15', axis=1), df['ARR_DEL15'], test_size=0.2, random_state=42)

A primeira instrução importa a função auxiliar [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) do Scikit-learn. A segunda linha usa a função para dividir o DataFrame em um conjunto de treinamento que contém a 80% dos dados originais e um conjunto de teste que contém os 20% restantes. O parâmetro random_state propaga o gerador de número aleatório usado para fazer a divisão, enquanto o primeiro e o segundo parâmetros são DataFrames que contêm as colunas de recurso e a coluna de rótulo.

` train_test_split ` retorna quatro DataFrames. Use o comando a seguir para exibir o número de linhas e colunas no DataFrame que contém as colunas de recurso usadas para treinamento:

```python
print('Ola')
```